# Epidemias

Em cada um dos casos
1. M/M/1: ρ = 0.5, λ = 1, μ = 2
2. M/M/1: ρ = 0.5, λ = 2, μ = 4
3. M/M/1: ρ = 2, λ = 4, μ = 2

avalie qual a fração de árvores finitas?

Em particular, relacione s = G(s)

In [88]:
import numpy as np
from collections import Counter
from scipy.stats import t

In [89]:
def simulador(lambda_=1, mi_=2, simulation_total_time=10000, deterministic=False):
    n = tempo_simulacao = num_chegadas = tempo_ocupado_servidor = 0
    clientes_fila, chegadas, freq_clientes, tempo_clientes_fila, tempo_clientes_sistema, tempo_espera_sistema = [], [], [], [], [], []
    
    maximo_de_iteracoes = 1000  # Definindo um limite de iterações para detectar se a árvore é finita ou infinita
    iteracao_atual = 0
    s_atual = None
    num_arvores_finitas = 0
    num_arvores_totais = 0

    exponential = (lambda mi: 1 / mi) if deterministic else (lambda mi: np.random.exponential(scale=1 / mi))

    while tempo_simulacao < simulation_total_time:
        tempo_chegada = np.random.exponential(scale=1 / lambda_)
        tempo_saida = exponential(mi_)
        last_event_time = tempo_simulacao

        if n == 0 or tempo_chegada < tempo_saida:
            if n == 0:
                freq_clientes.append(len(clientes_fila))

            tempo_simulacao += tempo_chegada
            num_chegadas += 1
            tempo_clientes_sistema.append((tempo_simulacao - last_event_time) * n)
            tempo_clientes_fila.append((tempo_simulacao - last_event_time) * len(clientes_fila))
            chegadas.append(tempo_simulacao)
            n += 1

            if n > 1:
                clientes_fila.append(tempo_simulacao)
                tempo_ocupado_servidor += tempo_chegada
                freq_clientes.append(len(clientes_fila))

        else:
            tempo_simulacao += tempo_saida
            tempo_clientes_sistema.append((tempo_simulacao - last_event_time) * n)
            tempo_clientes_fila.append((tempo_simulacao - last_event_time) * len(clientes_fila))

            if len(clientes_fila):
                clientes_fila.pop(0)

            tempo_espera_sistema.append(tempo_simulacao - chegadas.pop(0))
            tempo_ocupado_servidor += tempo_saida  
            n -= 1

        # Verificando se a árvore é finita
        iteracao_atual += 1
        if iteracao_atual == maximo_de_iteracoes:
            s_atual = len(clientes_fila)
        elif iteracao_atual > maximo_de_iteracoes and len(clientes_fila) == s_atual:
            num_arvores_finitas += 1
        
        num_arvores_totais += 1
    
    proporcao_arvores_finitas = num_arvores_finitas / num_arvores_totais
    # Calculando intervalo de confiança usando a distribuição t de Student
    graus_de_liberdade = num_arvores_totais - 1
    intervalo_confianca = t.interval(0.95, df=graus_de_liberdade, loc=proporcao_arvores_finitas, scale=np.sqrt((1 / num_arvores_totais) * proporcao_arvores_finitas * (1 - proporcao_arvores_finitas)))


    soma_tempo_clientes_fila = sum(tempo_clientes_fila)
    soma_tempo_clientes_sistema = sum(tempo_clientes_sistema)

    Wq = soma_tempo_clientes_fila / num_chegadas
    W = soma_tempo_clientes_sistema / num_chegadas
    L = soma_tempo_clientes_sistema / tempo_simulacao
    Lq = soma_tempo_clientes_fila / tempo_simulacao
    rho = tempo_ocupado_servidor / tempo_simulacao  
    pi = Counter(freq_clientes)

    for valor, freq in pi.items():
        pi[valor] = round(freq / num_chegadas, 6)

    

    results = {
        'Total de Clientes': num_chegadas,
        'Clientes Restantes': n,
        'Tempo Total': tempo_simulacao,
        'Tempo Espera Sistema': tempo_espera_sistema,
        'Wq': Wq,
        'W': W,
        'L': L,
        'Lq': Lq,
        'rho': rho,
        'pi': pi,
        'Proporção de Árvores Finitas': proporcao_arvores_finitas,
        'Intervalo de Confiança': intervalo_confianca
    }

    return results


In [91]:
#Cenários (λ, μ)
cenarios = [
    {'λ': 1, 'μ': 10},
    {'λ': 2, 'μ': 20},
    {'λ': 12, 'μ': 10},
]

for cenario in cenarios:
    print(f"----- Cenário: λ = {cenario['λ']}, μ = {cenario['μ']} -----")

    # Resultados da simulação
    result = simulador(lambda_=cenario['λ'], mi_=cenario['μ'], simulation_total_time=10000, deterministic=False)
    # for key, value in result.items():
    #     print(f"{key}: {value}")

    # Proporção esperada de árvores finitas (de acordo com a heurística)
    proporcao_esperada = cenario['λ'] / (cenario['λ'] + cenario['μ'])

    # Proporção obtida na simulação
    proporcao_obtida = result['Proporção de Árvores Finitas']

    print(f"Proporção Esperada de Árvores Finitas: {proporcao_esperada:.6f}")
    print(f"Proporção Obtida na Simulação: {proporcao_obtida:.6f}")

----- Cenário: λ = 1, μ = 10 -----
Proporção Esperada de Árvores Finitas: 0.090909
Proporção Obtida na Simulação: 0.901207
----- Cenário: λ = 2, μ = 20 -----
Proporção Esperada de Árvores Finitas: 0.090909
Proporção Obtida na Simulação: 0.922003
----- Cenário: λ = 12, μ = 10 -----
Proporção Esperada de Árvores Finitas: 0.545455
Proporção Obtida na Simulação: 0.000032


In [70]:
import numpy as np
from collections import Counter

def mm1_analytical(ρ):
    L = ρ / (1 - ρ)
    Lq = ρ**2 / (1 - ρ)
    W = 1 / (2 - ρ)
    Wq = ρ / (2 * (2 - ρ))
    return {'L': L, 'Lq': Lq, 'W': W, 'Wq': Wq, 'ρ': ρ}

# Cenários (λ, μ e ρ)
cenarios = [
    {'λ': 1, 'μ': 2, 'ρ': 0.5},
    {'λ': 2, 'μ': 4, 'ρ': 0.5},
    {'λ': 4, 'μ': 2, 'ρ': 2},
]

for cenario in cenarios:
    print(f"----- Cenário: λ = {cenario['λ']}, μ = {cenario['μ']}, ρ = {cenario['ρ']} -----")
    
    result = simulador(lambda_=cenario['λ'], mi_=cenario['μ'], simulation_total_time=10000, deterministic=False)

    # Resultados da simulação
    # for key, value in result.items():
    #     print(f"{key}: {value}")

    # Proporção esperada de árvores finitas (de acordo com a heurística)
    proporcao_esperada = cenario['λ'] / (cenario['λ'] + cenario['μ'])

    # Proporção obtida na simulação
    proporcao_obtida = result['Proporção de Árvores Finitas']

    print(f"Proporção Esperada de Árvores Finitas: {proporcao_esperada:.6f}")
    print(f"Proporção Obtida na Simulação: {proporcao_obtida:.6f}")



----- Cenário: λ = 1, μ = 2, ρ = 0.5 -----
Proporção Esperada de Árvores Finitas: 0.333333
Proporção Obtida na Simulação: 0.180552
----- Cenário: λ = 2, μ = 4, ρ = 0.5 -----
Proporção Esperada de Árvores Finitas: 0.333333
Proporção Obtida na Simulação: 0.604859
----- Cenário: λ = 4, μ = 2, ρ = 2 -----
Proporção Esperada de Árvores Finitas: 0.666667
Proporção Obtida na Simulação: 0.000183


### Cenário 1:  $λ= 1, μ= 2, ρ= 0.5$

In [83]:
result = simulador(lambda_=12, mi_=10, simulation_total_time=10000, deterministic=False)
print(result['Proporção de Árvores Finitas'])

6.817964882935544e-05


### Cenário 2:  $λ= 2, μ= 4, ρ= 0.5$

In [52]:
result = simulador(lambda_=2, mi_=4, simulation_total_time=10000, deterministic=False)
print(result['Proporção de Árvores Finitas'])

0.6172327044025158


### Cenário 2:  $λ= 4, μ= 2, ρ= 2$

In [60]:
result = simulador(lambda_=4, mi_=2, simulation_total_time=10000, deterministic=False)
print(result['Proporção de Árvores Finitas'])

4.9896049896049896e-05
